In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
cd/users/prave/datasets/csv files/competition/email cta

C:\users\prave\datasets\csv files\competition\email cta


In [3]:
train=pd.read_csv("train_F3fUq2S.csv")
test=pd.read_csv('test_Bk2wfZ3.csv')
sub=pd.read_csv('sample_submission_LJ2N3ZQ.csv')

In [4]:
train.head()

,campaign_id,sender,subject_len,body_len,mean_paragraph_len,day_of_week,is_weekend,times_of_day,category,product,...,is_image,is_personalised,is_quote,is_timer,is_emoticons,is_discount,is_price,is_urgency,target_audience,click_rate
0,1,3,76,10439,39,5,1,Noon,6,26,...,0,0,0,0,0,0,0,0,14,0.103079
1,2,3,54,2570,256,5,1,Morning,2,11,...,0,0,0,0,0,0,0,0,10,0.700000
2,3,3,59,12801,16,5,1,Noon,2,11,...,1,0,1,0,0,0,0,0,16,0.002769
3,4,3,74,11037,30,4,0,Evening,15,9,...,0,0,0,0,0,0,0,0,10,0.010868
4,5,3,80,10011,27,5,1,Noon,6,26,...,0,0,1,0,0,0,0,0,14,0.142826


In [5]:
train.dtypes

campaign_id             int64
sender                  int64
subject_len             int64
body_len                int64
mean_paragraph_len      int64
day_of_week             int64
is_weekend              int64
times_of_day           object
category                int64
product                 int64
no_of_CTA               int64
mean_CTA_len            int64
is_image                int64
is_personalised         int64
is_quote                int64
is_timer                int64
is_emoticons            int64
is_discount             int64
is_price                int64
is_urgency              int64
target_audience         int64
click_rate            float64
dtype: object

In [6]:
train['is_image'].value_counts()/len(train)

0    0.379237
1    0.362818
2    0.234110
3    0.020657
6    0.002119
4    0.001059
Name: is_image, dtype: float64

In [7]:
# Removing "is_timer","is_price" because they are constant variable
train.drop(['is_timer','is_price','campaign_id'],axis=1,inplace=True)
test.drop(['is_timer','is_price','campaign_id'],axis=1,inplace=True)

In [8]:
train['times_of_day']=train['times_of_day'].map({'Morning':1,'Noon':2,'Evening':3})
test['times_of_day']=test['times_of_day'].map({'Morning':1,'Noon':2,'Evening':3})

In [9]:
from itertools import combinations
from sklearn.preprocessing import LabelEncoder

def frequency_encoding(column_name,output_column_name,df):
    fe_pol=(df.groupby(column_name).size()/len(df))
    df[output_column_name]=df[column_name].apply(lambda x:fe_pol[x])
le=LabelEncoder()
le_features=[]
cat_features=[]

columns=['sender','target_audience','product','no_of_CTA','subject_len']


comb = combinations(columns, 2) 

for i in list(comb):  
    train[f'{i[0]}_{i[1]}']=train[i[0]].astype(str)+'_'+train[i[1]].astype(str)
    train[f'{i[0]}_{i[1]}_le']=le.fit_transform(train[f'{i[0]}_{i[1]}'])
    frequency_encoding(f'{i[0]}_{i[1]}',f'{i[0]}_{i[1]}',train)
    cat_features.append(f'{i[0]}_{i[1]}')  

frequency_encoding('sender','sender_fe',train)
frequency_encoding('subject_len','subject_len_fe',train)
frequency_encoding('product','product_fe',train)
frequency_encoding('target_audience','target_audience_fe',train)
frequency_encoding('category','category_fe',train)
frequency_encoding('no_of_CTA','no_of_CTA_fe',train)

In [10]:
from itertools import combinations
from sklearn.preprocessing import LabelEncoder

def frequency_encoding(column_name,output_column_name,df):
    fe_pol=(df.groupby(column_name).size()/len(df))
    df[output_column_name]=df[column_name].apply(lambda x:fe_pol[x])
le=LabelEncoder()
le_features=[]
cat_features=[]

columns=['sender','target_audience','product','no_of_CTA','subject_len']


comb = combinations(columns, 2) 

for i in list(comb):  
    test[f'{i[0]}_{i[1]}']=test[i[0]].astype(str)+'_'+test[i[1]].astype(str)
    test[f'{i[0]}_{i[1]}_le']=le.fit_transform(test[f'{i[0]}_{i[1]}'])
    frequency_encoding(f'{i[0]}_{i[1]}',f'{i[0]}_{i[1]}',test)
    cat_features.append(f'{i[0]}_{i[1]}')  

frequency_encoding('sender','sender_fe',test)
frequency_encoding('subject_len','subject_len_fe',test)
frequency_encoding('product','product_fe',test)
frequency_encoding('target_audience','target_audience_fe',test)
frequency_encoding('category','category_fe',test)
frequency_encoding('no_of_CTA','no_of_CTA_fe',test)

In [11]:
x=train.drop('click_rate',axis=1)
y=train['click_rate']

In [12]:
X_train=train.drop('click_rate',axis=1)
y_train=train['click_rate']
X_test=test

# Modeling

In [13]:
from xgboost import  XGBRegressor
from lightgbm import  LGBMRegressor
from catboost import  CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor,StackingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import xgboost as xgb
import lightgbm as lgb
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True).get_n_splits(y_train.values)
    r2= cross_val_score(model, X_train, y_train, scoring  = 'neg_root_mean_squared_error', cv = kf)
    return(r2)

model_xgb = xgb.XGBRegressor()
model_lgb = lgb.LGBMRegressor()
model_cb  = CatBoostRegressor(verbose=False)
model_rf  = RandomForestRegressor()
model_dt=DecisionTreeRegressor()

In [14]:
score=rmsle_cv(model_xgb)
xgb=XGBRegressor()
xgb.fit(X_train,y_train)
xgb_pred=xgb.predict(X_test)
print("XGBoost score:{:.4f} ({:.4f})\n".format(score.mean(),score.std()))
print("R2 score :{:.4f}".format(r2_score(xgb.predict(X_train),y_train)))
print("mse score :{:.4f}".format(mean_squared_error(xgb.predict(X_train),y_train)))

XGBoost score:-0.0770 (0.0184)

R2 score :0.9942
mse score :0.0000


In [15]:
score=rmsle_cv(model_rf)
rf=RandomForestRegressor()
rf.fit(X_train,y_train)
rf_pred=rf.predict(X_test)
print("RF score:{:.4f} ({:.4f})\n".format(score.mean(),score.std()))
print("R2 score :{:.4f}".format(r2_score(rf.predict(X_train),y_train)))
print("mse score :{:.4f}".format(mean_squared_error(rf.predict(X_train),y_train)))

RF score:-0.0759 (0.0185)

R2 score :0.9004
mse score :0.0005


In [35]:
score=rmsle_cv(model_lgb)
lgb=LGBMRegressor(learning_rate=0.35,boosting='dart',num_iteration=300,num_leaves=70)
lgb.fit(X_train,y_train)
lgb_pred=lgb.predict(X_test)
print("LGB score: {:.4f} ({:.4f})".format(score.mean(),score.std()))
print("R2 score :{:.4f}".format(r2_score(lgb.predict(X_train),y_train)))
print("mse score :{:.4f}".format(mean_squared_error(lgb.predict(X_train),y_train)))

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] num_iterations is set=300, num_iteration=300 will be ignored. Current value: num_iterations=300
LGB score: -0.0720 (0.0188)
R2 score :0.9718
mse score :0.0002


In [17]:
score=rmsle_cv(model_cb)
cb=CatBoostRegressor(eval_metric='R2',learning_rate=0.17)
cb.fit(X_train,y_train)
cb_pred=cb.predict(X_test)
print("cat score: {:.4f} ({:.4f})".format(score.mean(),score.std()))
print("R2 score :{:.4f}".format(r2_score(cb.predict(X_train),y_train)))
print("mse score :{:.4f}".format(mean_squared_error(cb.predict(X_train),y_train)))

0:	learn: 0.0697050	total: 19.8ms	remaining: 19.8s
1:	learn: 0.1411685	total: 25.6ms	remaining: 12.8s
2:	learn: 0.2126658	total: 31.2ms	remaining: 10.4s
3:	learn: 0.2607381	total: 36.7ms	remaining: 9.15s
4:	learn: 0.2976005	total: 43.1ms	remaining: 8.57s
5:	learn: 0.3258791	total: 49.3ms	remaining: 8.17s
6:	learn: 0.3498455	total: 55.6ms	remaining: 7.89s
7:	learn: 0.3779012	total: 63.1ms	remaining: 7.83s
8:	learn: 0.4091966	total: 69.5ms	remaining: 7.65s
9:	learn: 0.4304904	total: 74.9ms	remaining: 7.42s
10:	learn: 0.4394226	total: 79.7ms	remaining: 7.17s
11:	learn: 0.4670362	total: 85.6ms	remaining: 7.05s
12:	learn: 0.4905653	total: 91.8ms	remaining: 6.97s
13:	learn: 0.4969964	total: 96.7ms	remaining: 6.81s
14:	learn: 0.5158581	total: 102ms	remaining: 6.69s
15:	learn: 0.5194198	total: 107ms	remaining: 6.57s
16:	learn: 0.5379756	total: 112ms	remaining: 6.46s
17:	learn: 0.5475156	total: 117ms	remaining: 6.37s
18:	learn: 0.5558730	total: 121ms	remaining: 6.26s
19:	learn: 0.5696711	total:

182:	learn: 0.9195645	total: 878ms	remaining: 3.92s
183:	learn: 0.9197458	total: 883ms	remaining: 3.92s
184:	learn: 0.9204426	total: 888ms	remaining: 3.91s
185:	learn: 0.9208809	total: 893ms	remaining: 3.91s
186:	learn: 0.9212292	total: 897ms	remaining: 3.9s
187:	learn: 0.9221231	total: 902ms	remaining: 3.9s
188:	learn: 0.9224850	total: 906ms	remaining: 3.89s
189:	learn: 0.9232666	total: 911ms	remaining: 3.88s
190:	learn: 0.9242281	total: 916ms	remaining: 3.88s
191:	learn: 0.9246455	total: 920ms	remaining: 3.87s
192:	learn: 0.9248616	total: 924ms	remaining: 3.86s
193:	learn: 0.9251948	total: 928ms	remaining: 3.85s
194:	learn: 0.9254381	total: 932ms	remaining: 3.85s
195:	learn: 0.9254564	total: 937ms	remaining: 3.84s
196:	learn: 0.9265092	total: 941ms	remaining: 3.83s
197:	learn: 0.9272919	total: 945ms	remaining: 3.83s
198:	learn: 0.9275712	total: 949ms	remaining: 3.82s
199:	learn: 0.9278415	total: 953ms	remaining: 3.81s
200:	learn: 0.9282983	total: 957ms	remaining: 3.8s
201:	learn: 0.9

363:	learn: 0.9696816	total: 1.75s	remaining: 3.05s
364:	learn: 0.9698548	total: 1.75s	remaining: 3.05s
365:	learn: 0.9700257	total: 1.76s	remaining: 3.04s
366:	learn: 0.9701172	total: 1.76s	remaining: 3.04s
367:	learn: 0.9702035	total: 1.77s	remaining: 3.04s
368:	learn: 0.9703172	total: 1.77s	remaining: 3.03s
369:	learn: 0.9703204	total: 1.78s	remaining: 3.03s
370:	learn: 0.9704384	total: 1.78s	remaining: 3.03s
371:	learn: 0.9706450	total: 1.79s	remaining: 3.02s
372:	learn: 0.9709162	total: 1.79s	remaining: 3.02s
373:	learn: 0.9710309	total: 1.8s	remaining: 3.01s
374:	learn: 0.9712042	total: 1.8s	remaining: 3s
375:	learn: 0.9713500	total: 1.81s	remaining: 3s
376:	learn: 0.9714365	total: 1.81s	remaining: 3s
377:	learn: 0.9715630	total: 1.82s	remaining: 2.99s
378:	learn: 0.9716993	total: 1.82s	remaining: 2.98s
379:	learn: 0.9718387	total: 1.83s	remaining: 2.98s
380:	learn: 0.9719918	total: 1.83s	remaining: 2.98s
381:	learn: 0.9721425	total: 1.83s	remaining: 2.97s
382:	learn: 0.9723706	t

543:	learn: 0.9856141	total: 2.6s	remaining: 2.18s
544:	learn: 0.9857039	total: 2.61s	remaining: 2.18s
545:	learn: 0.9857637	total: 2.62s	remaining: 2.17s
546:	learn: 0.9858605	total: 2.62s	remaining: 2.17s
547:	learn: 0.9859254	total: 2.63s	remaining: 2.17s
548:	learn: 0.9859811	total: 2.63s	remaining: 2.16s
549:	learn: 0.9860153	total: 2.64s	remaining: 2.16s
550:	learn: 0.9860835	total: 2.64s	remaining: 2.15s
551:	learn: 0.9861282	total: 2.65s	remaining: 2.15s
552:	learn: 0.9861664	total: 2.65s	remaining: 2.14s
553:	learn: 0.9862107	total: 2.66s	remaining: 2.14s
554:	learn: 0.9862617	total: 2.66s	remaining: 2.13s
555:	learn: 0.9863178	total: 2.67s	remaining: 2.13s
556:	learn: 0.9863690	total: 2.67s	remaining: 2.12s
557:	learn: 0.9863875	total: 2.67s	remaining: 2.12s
558:	learn: 0.9864176	total: 2.68s	remaining: 2.11s
559:	learn: 0.9864559	total: 2.68s	remaining: 2.11s
560:	learn: 0.9864833	total: 2.69s	remaining: 2.1s
561:	learn: 0.9865445	total: 2.69s	remaining: 2.1s
562:	learn: 0.9

728:	learn: 0.9925822	total: 3.5s	remaining: 1.3s
729:	learn: 0.9925916	total: 3.51s	remaining: 1.3s
730:	learn: 0.9926093	total: 3.51s	remaining: 1.29s
731:	learn: 0.9926294	total: 3.52s	remaining: 1.29s
732:	learn: 0.9926425	total: 3.52s	remaining: 1.28s
733:	learn: 0.9926563	total: 3.53s	remaining: 1.28s
734:	learn: 0.9926934	total: 3.54s	remaining: 1.27s
735:	learn: 0.9927143	total: 3.54s	remaining: 1.27s
736:	learn: 0.9927434	total: 3.54s	remaining: 1.26s
737:	learn: 0.9927616	total: 3.55s	remaining: 1.26s
738:	learn: 0.9927967	total: 3.55s	remaining: 1.25s
739:	learn: 0.9928002	total: 3.56s	remaining: 1.25s
740:	learn: 0.9928259	total: 3.56s	remaining: 1.25s
741:	learn: 0.9928398	total: 3.57s	remaining: 1.24s
742:	learn: 0.9928614	total: 3.57s	remaining: 1.24s
743:	learn: 0.9928677	total: 3.58s	remaining: 1.23s
744:	learn: 0.9928812	total: 3.58s	remaining: 1.23s
745:	learn: 0.9929036	total: 3.58s	remaining: 1.22s
746:	learn: 0.9929414	total: 3.59s	remaining: 1.22s
747:	learn: 0.9

914:	learn: 0.9956129	total: 4.34s	remaining: 403ms
915:	learn: 0.9956362	total: 4.34s	remaining: 398ms
916:	learn: 0.9956551	total: 4.35s	remaining: 394ms
917:	learn: 0.9956554	total: 4.35s	remaining: 389ms
918:	learn: 0.9956811	total: 4.36s	remaining: 384ms
919:	learn: 0.9956964	total: 4.36s	remaining: 379ms
920:	learn: 0.9957228	total: 4.37s	remaining: 375ms
921:	learn: 0.9957233	total: 4.37s	remaining: 370ms
922:	learn: 0.9957362	total: 4.38s	remaining: 365ms
923:	learn: 0.9957545	total: 4.38s	remaining: 360ms
924:	learn: 0.9957583	total: 4.38s	remaining: 356ms
925:	learn: 0.9957703	total: 4.39s	remaining: 351ms
926:	learn: 0.9957910	total: 4.39s	remaining: 346ms
927:	learn: 0.9957941	total: 4.4s	remaining: 341ms
928:	learn: 0.9958005	total: 4.4s	remaining: 336ms
929:	learn: 0.9958073	total: 4.41s	remaining: 332ms
930:	learn: 0.9958122	total: 4.41s	remaining: 327ms
931:	learn: 0.9958190	total: 4.41s	remaining: 322ms
932:	learn: 0.9958336	total: 4.42s	remaining: 317ms
933:	learn: 0.

In [36]:
sub=pd.DataFrame({'campaign_id':sub['campaign_id'],'click_rate':lgb_pred})
sub.to_csv('lgbm97.csv',index=False)

In [37]:
sub

,campaign_id,click_rate
0,1889,0.031324
1,1890,0.506044
2,1891,0.172030
3,1892,0.183268
4,1893,0.117619
...,...,...
757,2646,0.013529
758,2647,0.052510
759,2648,0.051129
760,2649,0.268682
